
# Export a function as FMU


``otfmi.FunctionExporter`` enables to export OpenTURNS functions as FMUs. The
main interest of this class is to use OpenTURNS metamodels in a simulation environment.

Currently, the inclusion of a metamodel in
[OpenModelica GUI](https://openmodelica.org/?id=78:omconnectioneditoromedit&catid=10:main-category)
has been performed once (see
[this paper](https://www.researchgate.net/publication/354810878_Analysis_and_reduction_of_models_using_Persalys)).



------------

## FMU generation

First, we create the OpenTURNS function to export as FMU.
This example being solely a demonstrator, we consider a very simple
exponential function.



In [ ]:
import openturns as ot
import otfmi
import tempfile
from os.path import join

func = ot.SymbolicFunction(["x"], ["exp(x)"])
inputPoint = [2.0]
print(func(inputPoint))

We create the FMU constructor and the folder in which save the FMU:



In [ ]:
fmuExporter = otfmi.FunctionExporter(func)
fmu_path = join(tempfile.mkdtemp(), "myExponential.fmu")
print(fmu_path)

We export the OpenTURNS function as a Model Exchange FMU.
The counterpart is to export as CoSimulation FMU (embeds a solver).
Another option would be to export the function via pythonfmu (see the "mode" keyword)
allowing export of temporal functions (not a concern here for our function).



In [ ]:
fmuExporter.export_fmu(fmu_path, fmuType="me")

<div class="alert alert-info"><h4>Note</h4><p>The export requires [CMake](https://cmake.org/) build system, a C compiler (Visual Studio or
  GCC for instance), and the OpenModelica compiler
  [OMC](https://www.openmodelica.org/?id=51:open-modelica-compiler-omccecatid=10:main-category).</p></div>



Simple as it looks, this function actually does the following :

- write a C-wrapper for the OpenTURNS function,
- write a Modelica model calling the C-wrapper as [External](shorturl.at/fhCU2) function,
- export the Modelica model as FMU using OpenModelica compiler.

<div class="alert alert-info"><h4>Note</h4><p>To retrieve only the Modelica model, see :doc:`here <ex_model_exporter>`.</p></div>

<div class="alert alert-danger"><h4>Warning</h4><p>The path to the C-wrapper is hard-coded in the model & in the FMU.</p></div>

----------------------

## FMU validation

We import the in OpenModelica GUI. We connect the wrapper to an input sine signal
(*Modelica.Blocks.Sources.Sine*) and to an output block (*Modelica.Blocks.Interfaces.RealOutput*):

<img src="file://_static/TestMyExponentialFMU.png" scale="100 %" alt="alternate text" align="center">

We simulate the model on 1 second, with 50 time steps. We can verify that `y`
output corresponds to the exponential of the sine signal.

<img src="file://_static/show_output.png" scale="80 %" alt="alternate text" align="center">

<div class="alert alert-danger"><h4>Warning</h4><p>Compared to native Modelica functions, the included OpenTURNS function is
   slow. Using the FMU, 11 seconds of simulation were required for 50 time
   steps (i.e. 50 function calls).</p></div>

